In [1]:
import pandas as pd
from edc import optimisation
from edc import utils

%load_ext autoreload
%autoreload 2

In [2]:
df_routes = pd.read_csv("../data/routes_v2.csv")

In [3]:
df_routes["stops_list"] = df_routes.apply(lambda row : row.stops.split(' > '),axis=1)

In [22]:
def best_traject(row):
    list_of_cities_with_coord = [utils.donnees_cities_with_name(ville) for ville in row.stops_list]
    try :
        return optimisation.optimiser_trajectoire(list_of_cities_with_coord)
    except :
        if len(row.stops_list) == 1 :
            return row.stops_list
        else :
            print(row.stops_list)
            raise ZeroDivisionError

In [13]:
len(df_routes)

7426

In [23]:
# df_routes['trajectoires_optimisees'] = df_routes.apply(best_traject ,axis=1)
# df_routes.to_csv("..\data\intermediate_data\df_routes_optimisees.csv")

In [24]:
df_routes_optimisees = pd.read_csv("..\data\intermediate_data\df_routes_optimisees.csv")

In [25]:
df_routes_optimisees

,Unnamed: 0,truck_id,duration,fill_volume,n_units,orders,from_warehouse,route_date,stops,total_distance,stops_list,trajectoires_optimisees
0,0,jehwGBpNg2rDLeKoYeiaLs,2.167136,5.9879,87.0,nK4iPf8nrGA9yivRH66ehS > jnsH9ndQ3UKcWSbb2UMkFn,Clermont-Ferrand,2021-01-02,Clermont-Ferrand > Bourges,153.950611,"['Clermont-Ferrand', 'Bourges']","['Clermont-Ferrand', 'Bourges']"
1,1,XnzFnkALW3tLnzrdCVFyKH,23.230412,4.7060,190.0,XHfAWaLxTFbo5UQwtd7CMz > Wd6rPc4wPRNmt3ruf3F7o...,Cergy,2021-01-02,Cergy > Saint-Nazaire > Saint-Denis > Rouen > ...,1416.184102,"['Cergy', 'Saint-Nazaire', 'Saint-Denis', 'Rou...","['Cergy', 'Boulogne-sur-Mer', 'Rouen', 'Caen',..."
2,2,dwQ5Xd6TwcXURBczFdRXVy,6.995822,10.9609,148.0,gQi59MNfDa5YFpVWxfmPin > FaqH6N2zWCeykLh5f9VPW...,Cergy,2021-01-02,Cergy > Boulogne-Billancourt > Aubervilliers >...,184.704129,"['Cergy', 'Boulogne-Billancourt', 'Aubervillie...","['Cergy', 'Boulogne-Billancourt', 'Antony', 'A..."
3,3,A58tMgZVPfe2ucEBcCSXTW,4.785198,1.8904,79.0,7bzBdqUKWXyNrAhntXNrBq > BZsYUHDe4eVj7HaeM8MBt7,Reims,2021-01-02,Reims > Épinal > Troyes,390.851862,"['Reims', 'Épinal', 'Troyes']","['Reims', 'Troyes', 'Épinal']"
4,4,KRNfDAmUV8bRzkxhdAYfkj,4.413343,0.4536,63.0,DfN8XqUVeVHjTW3Pat76Tr > MUT2TaTikTsi8FzSCnimxv,Clermont-Ferrand,2021-01-03,Clermont-Ferrand > Villeurbanne,139.447356,"['Clermont-Ferrand', 'Villeurbanne']","['Clermont-Ferrand', 'Villeurbanne']"
...,...,...,...,...,...,...,...,...,...,...,...,...
7421,7421,4q43tx2jAeJtMFz4UwGZpD,1.150110,68.8800,41.0,JXpxeURRDzSAgeE5FiPh7H,Montauban,2021-06-29,Montauban > Pessac,180.534974,"['Montauban', 'Pessac']","['Montauban', 'Pessac']"
7422,7422,kRYyBXN9TeJF8Mewj55RdU,8.596979,69.6117,95.0,EpgfunMiFtk37q3hniS8Te > Cujg8Rgb8uvYWtSzA4y4a...,Montauban,2021-06-29,Montauban > Pessac > Perpignan > Pau,821.990387,"['Montauban', 'Pessac', 'Perpignan', 'Pau']","['Montauban', 'Perpignan', 'Pau', 'Pessac']"
7423,7423,9LBChivrfyZZrBMpcjLtGv,21.367729,49.4855,307.0,bRay4XPtfJtqcfu9PcfHUN > Tn3ehPUBELhd3JxMPSD6f...,Montauban,2021-06-29,Montauban > Niort > Mérignac > Montauban > Lim...,1034.651961,"['Montauban', 'Niort', 'Mérignac', 'Montauban'...","['Montauban', 'Montauban', 'Limoges', 'Niort',..."
7424,7424,YjL2avoUgG3tdDxditdH5V,1.651791,79.3388,41.0,S254SKuGaFfScnCCyAUfWk > 9pvVqJzgLFF2PcypNjAc5V,Montauban,2021-06-29,Montauban > La Rochelle,308.878633,"['Montauban', 'La Rochelle']","['Montauban', 'La Rochelle']"
